In [0]:
"""
Credit breached report
For user_id 1, Moustafa has 100 but pays to Winston 400 and receves 200 from Jonathan. 
1 -> [100 - 400 + 200] --> -100 so credit breached will be Yes
Input
+-------+---------+------+
|user_id|user_name|credit|
+-------+---------+------+
|      1| Moustafa|   100|
|      2| Jonathan|   200|
|      3|  Winston| 10000|
|      4|     Luis|   800|
+-------+---------+------+

+--------+-------+-------+------+-------------+
|trans_id|paid_by|paid_to|amount|transacted_on|
+--------+-------+-------+------+-------------+
|       1|      1|      3|   400|   2020-08-01|
|       2|      3|      2|   500|   2020-08-02|
|       3|      2|      1|   200|   2020-08-03|
+--------+-------+-------+------+-------------+



Output
+---------+------+---------------------+
|user_name|credit|credit_limit_breached|
+---------+------+---------------------+
| Moustafa|  -100|                  Yes|
| Jonathan|   500|                   No|
|  Winston|  9900|                   No|
|     Luis|   800|                   No|
+---------+------+---------------------+
"""
users_df = spark.createDataFrame(
    [
        ('1', 'Moustafa', '100'),
        ('2', 'Jonathan', '200'),
        ('3', 'Winston', '10000'),
        ('4', 'Luis', '800')
    ], ["user_id", "user_name", "credit"]
)

transactions_df = spark.createDataFrame(
    [
        (1, 1, 3, 400, '2020-08-01'),
        (2, 3, 2, 500, '2020-08-02'),
        (3, 2, 1, 200, '2020-08-03')
    ], 
    ["trans_id", "paid_by", "paid_to", "amount", "transacted_on"]
) 

users_df.show()
transactions_df.show()


+-------+---------+------+
|user_id|user_name|credit|
+-------+---------+------+
|      1| Moustafa|   100|
|      2| Jonathan|   200|
|      3|  Winston| 10000|
|      4|     Luis|   800|
+-------+---------+------+

+--------+-------+-------+------+-------------+
|trans_id|paid_by|paid_to|amount|transacted_on|
+--------+-------+-------+------+-------------+
|       1|      1|      3|   400|   2020-08-01|
|       2|      3|      2|   500|   2020-08-02|
|       3|      2|      1|   200|   2020-08-03|
+--------+-------+-------+------+-------------+



In [0]:
from pyspark.sql.functions import *

users_df.join(transactions_df, users_df["user_id"] == transactions_df["paid_by"], "left") \
    .withColumn("debited_amount", -col("amount")) \
    .drop("amount", "trans_id", "paid_by", "paid_to", "transacted_on") \
    .join(transactions_df, users_df["user_id"] == transactions_df["paid_to"], "left") \
    .withColumn("credited_amount", col("amount")) \
    .drop("user_id","trans_id", "paid_by", "paid_to", "transacted_on") \
    .withColumn("credit", col("credit") + coalesce(col("debited_amount"), lit(0)) + coalesce(col("credited_amount"), lit(0))) \
    .withColumn("credit_limit_breached", when(col("credit") < lit(0), "Yes").otherwise(lit("No"))) \
    .select("user_name", "credit", "credit_limit_breached") \
    .show()


+---------+------+---------------------+
|user_name|credit|credit_limit_breached|
+---------+------+---------------------+
| Moustafa|  -100|                  Yes|
| Jonathan|   500|                   No|
|  Winston|  9900|                   No|
|     Luis|   800|                   No|
+---------+------+---------------------+



In [0]:
users_df.createOrReplaceTempView("users")
transactions_df.createOrReplaceTempView("transactions")

spark.sql("""
          with cte as (
            select 
                u.user_name, u.credit, -t.amount as debited_amount, t1.amount as credited_amount 
            from users u
            left join transactions t on u.user_id = t.paid_by
            left join transactions t1 on u.user_id = t1.paid_to
          ), cte2 as(
          select
            user_name, credit + coalesce(debited_amount, 0) + coalesce(credited_amount, 0) as credit
          from cte
          )
          select
            user_name, credit, 
            case when credit < 0 then "Yes" else "No" end as credit_limit_breached
          from cte2
          """).show()


+---------+------+---------------------+
|user_name|credit|credit_limit_breached|
+---------+------+---------------------+
| Moustafa|  -100|                  Yes|
| Jonathan|   500|                   No|
|  Winston|  9900|                   No|
|     Luis|   800|                   No|
+---------+------+---------------------+

